In [1]:
#ライブラリ、モジュールのインポート
import os 
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torchvision.transforms import ToPILImage
from PIL import Image
import shutil
import uuid  # ユニークな識別子を生成するためのモジュール

In [2]:
#GPUの確認
import torch
if torch.cuda.is_available():
  print("GPUの設定ができています。")
else:
  print("GPUの設定ができていません！")

GPUの設定ができています。


In [39]:
# 画像データが格納されたディレクトリパス
image_data_path = "/home/yasumasa-tezuka/data_intern/train_data/images"
# テキストデータが格納されたディレクトリパス
text_data_path = "/home/yasumasa-tezuka/data_intern/train_data/labels"

data_transforms = transforms.Compose([
    transforms.RandomResizedCrop(640, scale=(0.8, 1.0)),  # ランダムにサイズ変更とクロップ
    transforms.RandomHorizontalFlip(p=0.5),  # ランダムな水平反転
    transforms.RandomVerticalFlip(p=0.5),  # ランダムな垂直反転
    transforms.RandomRotation(degrees=90),  # ランダムな回転（-90度から90度）
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # 色調の変更
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=15),  # アフィン変換
    transforms.RandomPerspective(distortion_scale=0.5, p=0.5, interpolation=3),  # ランダムな透視変換
    transforms.GaussianBlur(kernel_size=7, sigma=(0.1, 2.0)),  # ガウシアンぼかし
    transforms.ToTensor(),  # PyTorchのテンソルに変換
])


In [26]:
# トレーニングデータの新しいディレクトリパス
train_data_dir = "/home/yasumasa-tezuka/data_intern/train_data/images_expansion"

# ディレクトリが存在しない場合、作成
if not os.path.exists(train_data_dir):
    os.makedirs(train_data_dir)

# 画像データのリスト
image_files = glob.glob(os.path.join(image_data_path, "*.jpg"))  # 画像ファイルの拡張子に合わせて修正

# テキストファイルのリストを作成（対応するテキストファイルのファイル名を生成）
text_files = [os.path.join(text_data_path, os.path.splitext(os.path.basename(image_file))[0] + ".txt") for image_file in image_files]

# データ拡張を20回繰り返す
for i in range(20):
    # データ拡張とコピー
    for image_file in image_files:
        image = Image.open(image_file)
        # データ拡張を適用
        augmented_image = data_transforms(image)

        # TensorをPILイメージに変換
        augmented_image_pil = ToPILImage()(augmented_image)
        
        # 新しいファイル名を生成（例：元のファイル名 + ユニークな識別子 + 拡張子）
        file_name = os.path.basename(image_file)
        base_name_without_extension, extension = os.path.splitext(file_name)
        unique_id = str(uuid.uuid4())[:8]  # ランダムな8文字の識別子
        new_file_name = f"{base_name_without_extension}_{unique_id}{extension}"

        # トレーニングデータディレクトリに保存
        augmented_image_pil.save(os.path.join(train_data_dir, new_file_name))

        # 対応するテキストファイルの名前を生成
        text_file = os.path.join(text_data_path, base_name_without_extension + ".txt")
        new_text_file_name = os.path.basename(new_file_name)
        new_text_file_name = os.path.splitext(new_text_file_name)[0] + ".txt"

        # トレーニングデータディレクトリにコピー
        shutil.copy(text_file, os.path.join(train_data_dir, new_text_file_name))

    # データ拡張後のデータ数
    augmented_data_size = len(os.listdir(train_data_dir))

    print(f"データ拡張{i+1}回目後のデータ数:", augmented_data_size)


データ拡張1回目後のデータ数: 254
データ拡張2回目後のデータ数: 508
データ拡張3回目後のデータ数: 762
データ拡張4回目後のデータ数: 1016
データ拡張5回目後のデータ数: 1270
データ拡張6回目後のデータ数: 1524
データ拡張7回目後のデータ数: 1778
データ拡張8回目後のデータ数: 2032
データ拡張9回目後のデータ数: 2286
データ拡張10回目後のデータ数: 2540
データ拡張11回目後のデータ数: 2794
データ拡張12回目後のデータ数: 3048
データ拡張13回目後のデータ数: 3302
データ拡張14回目後のデータ数: 3556
データ拡張15回目後のデータ数: 3810
データ拡張16回目後のデータ数: 4064
データ拡張17回目後のデータ数: 4318
データ拡張18回目後のデータ数: 4572
データ拡張19回目後のデータ数: 4826
データ拡張20回目後のデータ数: 5080


In [63]:
#アノテーションした画像データの取得
data_path = "/home/yasumasa-tezuka/data_intern/train_data/images/"
data_path_1 = "/home/yasumasa-tezuka/data_intern/train_data/labels/"

x_list = glob.glob(data_path + "*.jpg") + glob.glob(data_path + "*.JPG")
y_list = glob.glob(data_path_1 + "*.txt")

if len(x_list) != len(y_list):
    print(f"警告: x_list と y_list の長さが一致していません（x_list: {len(x_list)}, y_list: {len(y_list)}）")
else:
    x_train, x_test, y_train, y_test = train_test_split(x_list, y_list)
    print("訓練用", len(x_train))
    print("検証用", len(x_test))


# リストが空でないか確認
if not x_list or not y_list:
    print("エラー: ファイルが見つかりません。")
else:
    x_train, x_test, y_train, y_test = train_test_split(x_list, y_list)
    print("訓練用", len(x_train))
    print("検証用", len(x_test))


訓練用 97
検証用 33
訓練用 97
検証用 33


In [ ]:
#あしたためす

In [ ]:
from sklearn.model_selection import train_test_split
import shutil
import os
from tqdm import tqdm

# 画像とラベルのディレクトリ
image_dir = "/path/to/images"
label_dir = "/path/to/labels"

# 画像ファイルのリスト
image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')] # 例として .jpg を使用

# 画像ファイルの名前（拡張子を除く）に基づいてラベルファイルをマッチング
data = [(os.path.join(image_dir, f), os.path.join(label_dir, f.replace('.jpg', '.txt'))) for f in image_files]

# 画像とラベルのペアをトレーニングと検証用に分割
train_data, val_data = train_test_split(data, test_size=0.2) # 例として20%を検証用に使用

# 新しいディレクトリを作成
for directory in [train_dir, val_dir]:
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory)

# トレーニングデータのコピー
for image, label in tqdm(train_data):
    shutil.copyfile(image, os.path.join(train_dir, os.path.basename(image)))
    shutil.copyfile(label, os.path.join(train_dir, os.path.basename(label)))

# 検証データのコピー
for image, label in tqdm(val_data):
    shutil.copyfile(image, os.path.join(val_dir, os.path.basename(image)))
    shutil.copyfile(label, os.path.join(val_dir, os.path.basename(label)))


In [61]:
# from tqdm import tqdm
# import shutil
# import os

# # トレーニング用と検証用ディレクトリのパス
# train_dir = "/home/yasumasa-tezuka/data_intern/train_data/images/train"
# val_dir = "/home/yasumasa-tezuka/data_intern/train_data/labels/val"

# # 既存のディレクトリを削除し、新しく作成
# for directory in [train_dir, val_dir]:
#     if os.path.exists(directory):
#         shutil.rmtree(directory)
#     os.makedirs(directory)

# # トレーニングデータのコピー
# for f in tqdm(x_train + y_train):
#     file_name = os.path.basename(f)
#     shutil.copyfile(f, os.path.join(train_dir, file_name))

# # 検証データのコピー
# for f in tqdm(x_test + y_test):
#     file_name = os.path.basename(f)
#     shutil.copyfile(f, os.path.join(val_dir, file_name))

100%|██████████████████████████████████████████| 66/66 [00:00<00:00, 988.83it/s]


In [29]:
#data.yamlの変更
yaml = """
train: /home/yasumasa-tezuka/data_intern/train_data/images_expansion/train
val: /home/yasumasa-tezuka/data_intern/train_data/images_expansion/val

nc: 1
names: ['tree']
"""

#data.yamlの作成または上書き保存
with open("data.yaml", "w") as f:
  f.write(yaml)

In [30]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
%pip install -qr requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 16083, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 16083 (delta 7), reused 12 (delta 1), pack-reused 16056
Receiving objects: 100% (16083/16083), 14.71 MiB | 7.86 MiB/s, done.
Resolving deltas: 100% (11035/11035), done.
/home/yasumasa-tezuka/NeuroDive_日揮インターン/intern_tree_number_github/tree_number_intern/intern_YOLO.v5/yolov5/yolov5/yolov5/yolov5/yolov5/yolov5
Note: you may need to restart the kernel to use updated packages.


In [62]:
# トレーニングの実行
!python train.py --img 640 --batch 16 --epochs 50 --data /home/yasumasa-tezuka/data_intern/1~50annotation/data.yaml --weights yolov5s.pt

2023-12-04 01:40:57.613391: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 01:40:57.613434: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 01:40:57.613468: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/home/yasumasa-tezuka/data_intern/1~50annotation/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, op

In [59]:
import cv2

# 画像を読み込む
image_path = '/home/yasumasa-tezuka/data_intern/train_data/images/0ace05a4-imori1.jpg'  # 画像の実際のファイルパスを指定してください
image = cv2.imread(image_path)

# クリックした座標を保持するリスト
click_coordinates = []

# クリックイベントを処理するコールバック関数
def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        click_coordinates.append((x, y))
        print(f"クリック座標: x={x}, y={y}")

# 画像ウィンドウを表示
cv2.imshow('Image', image)

# マウスクリックイベントを設定
cv2.setMouseCallback('Image', click_event)

# 画像ウィンドウが閉じられるまで待機
cv2.waitKey(0)
cv2.destroyAllWindows()

# クリックした座標を表示
print("クリックした座標:", click_coordinates)


error: OpenCV(4.8.1) /io/opencv/modules/highgui/src/window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [52]:
import cv2

# 画像を読み込む
image_path = '/home/yasumasa-tezuka/data_intern/train_data/images/0ace05a4-imori1.jpg'  # 画像の実際のファイルパスを指定してください
image = cv2.imread(image_path)

# クリックした座標を保持するリスト
click_coordinates = []

# クリックイベントを処理するコールバック関数
def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        click_coordinates.append((x, y))
        print(f"クリック座標: x={x}, y={y}")
print(image)
# 画像ウィンドウを表示
cv2.imshow('Image', image)

# マウスクリックイベントを設定
cv2.setMouseCallback('Image', click_event)

# 画像ウィンドウが閉じられるまで待機
cv2.waitKey(0)
cv2.destroyAllWindows()

# クリックした座標を表示
print("クリックした座標:", click_coordinates)


[[[204 170 140]
  [205 171 141]
  [207 173 143]
  ...
  [  0  13   2]
  [  0  17   6]
  [  6  23  12]]

 [[204 170 140]
  [205 171 141]
  [206 172 142]
  ...
  [  7  24  13]
  [ 12  29  18]
  [ 19  36  25]]

 [[204 170 140]
  [205 171 141]
  [206 172 142]
  ...
  [  5  22  11]
  [ 10  26  15]
  [ 16  32  21]]

 ...

 [[ 53  53  39]
  [ 53  53  39]
  [ 54  54  42]
  ...
  [ 67  84  87]
  [ 72  87  90]
  [ 67  82  85]]

 [[ 51  54  39]
  [ 52  55  40]
  [ 52  55  40]
  ...
  [ 68  83  86]
  [ 69  81  85]
  [ 61  73  77]]

 [[ 51  54  39]
  [ 52  55  40]
  [ 53  56  41]
  ...
  [ 72  87  90]
  [ 71  83  87]
  [ 61  73  77]]]


error: OpenCV(4.8.1) /io/opencv/modules/highgui/src/window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
